<a href="https://colab.research.google.com/github/vothane/Big-League-SQLew/blob/main/Big_Leagues.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install spark (change the version number if needed)
!wget -q https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz

# Unzip the spark file to the current folder
!tar xf spark-3.2.0-bin-hadoop3.2.tgz

# Set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

# Install findspark using pip
!pip install -q findspark

# Spark for Python
!pip install pyspark

     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 63.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=15f6ffe2243dbc2e3d371b4c49b7b587b02e850fcf435d948ac0e6d4f99fff16
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
class Table:
    def __init__(self):
        self.rows = []

    @property
    def get_rows(self):
        return self.rows

    def insert(self, row_values):
        self.rows.append(row_values)

class TableBuilder:
    @staticmethod
    def build_table(url):
        html_doc = requests.get(url)
        html_content = BeautifulSoup(html_doc.content, 'html.parser')
        
        raw = html_content.find('thead')
        data = raw.find_all('tr')
        rows = data[1:]

        table = Table()

        convert = lambda txt: float(txt) if txt.replace('.', '', 1).isdigit() else txt
        
        for cols in rows:
            col = cols.find_all("td")
            table.insert([convert(txt.text) for txt in col])

        return table        

In [ ]:
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# define schema for our data
schema = StructType([
   StructField("Pitch", StringType(), False),
   StructField("Count", FloatType(), False),
   StructField("Foul/Swing", FloatType(), False),
   StructField("Whiff/Swing", FloatType(), False),
   StructField("GB/BIP", FloatType(), False),
   StructField("LD/BIP", FloatType(), False),
   StructField("FB/BIP", FloatType(), False),
   StructField("PU/BIP", FloatType(), False),
   StructField("GB/FB", FloatType(), False),
   StructField("HR/(FB+LD)", FloatType(), False)])

url = "http://www.brooksbaseball.net/tabs.php?player=456034&var=so"
table = TableBuilder.build_table(url)

data = table.get_rows
print(data)

[['Fourseam', 8234.0, 44.58, 24.31, 38.02, 21.76, 30.91, 9.31, 123.01, 6.75], ['Sinker', 11284.0, 43.22, 16.5, 45.69, 21.59, 26.07, 6.65, 175.23, 7.22], ['Change', 5398.0, 30.26, 30.86, 45.23, 24.98, 22.62, 7.17, 200.0, 6.74], ['Slider', 609.0, 41.34, 20.49, 57.01, 16.82, 17.76, 8.41, 321.05, 21.62], ['Curve', 2750.0, 35.0, 26.75, 48.5, 20.32, 24.94, 6.24, 194.44, 10.71], ['Cutter', 4768.0, 41.21, 21.52, 44.51, 24.63, 24.51, 6.34, 181.59, 7.94]]


In [ ]:
from pyspark.sql.functions import col

spark = (SparkSession.builder.appName("Big_Leagues").getOrCreate())

sabermetrics = spark.createDataFrame(data, schema)
 
# show David Price most used pitches in descending order
(sabermetrics.select("Pitch")
             .where(col("Count") > 5000)
             .orderBy(desc("Count"))).show()

+--------+
|   Pitch|
+--------+
|  Sinker|
|Fourseam|
|  Change|
+--------+

